# Optimizing the model with series of experiments

In [ ]:
import pathlib

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_val_score

### Let´s first reload the data from the previous notebook. Then, perform train_test_split once again.

In [ ]:
DATA_DIR = pathlib.Path.cwd().parent / 'data'
print(DATA_DIR)

In [ ]:
model_data_scaled_path = DATA_DIR / 'processed' / 'ames_model_data_scaled.pkl'

In [ ]:
data = pd.read_pickle(model_data_scaled_path)

In [ ]:
X = data.drop(columns=['SalePrice']).copy().values
y = data['SalePrice'].copy().values

In [ ]:
X.shape, y.shape

In [ ]:
RANDOM_SEED = 42  # Any number here, really.

Xtrain, Xtest, ytrain, ytest = train_test_split(
    X,
    y,
    test_size=0.25,
    random_state=RANDOM_SEED,
)

### Experiment 1: LinearRegression with encoded categorical values AND scaled continuous variables

Our first experiment is to rerun the linear regression model with the same features as before, but this time on the new data with scaled continuous variables.

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
linear_scaled_model = LinearRegression()

linear_scaled_model.fit(Xtrain, ytrain)

In [ ]:
scores_linear_scaled = cross_val_score(linear_scaled_model,
                                       Xtrain,
                                       ytrain,
                                       cv=10,
                                       scoring='neg_mean_squared_error',
                                       n_jobs=-1)

scores_linear_scaled = np.sqrt(-scores_linear_scaled)
print(f'MEAN: {scores_linear_scaled.mean():.2f} +/- {scores_linear_scaled.std():.2f}')
error_percent_linear = 100 * (10**scores_linear_scaled.mean() - 1)
std_percent_linear = 100 * (10**scores_linear_scaled.std() - 1)
print(f'Average error is {error_percent_linear:.2f}%')
print(f'Standard deviation of error is {std_percent_linear:.2f}%')

path = DATA_DIR / 'processed' / 'models_log.csv'

with open(path, 'a') as f:
    f.write(f'Linear Regression Scaled,{error_percent_linear:.2f},{std_percent_linear:.2f}\n')

The experiment showed no signifficant difference in the model performance. This means that the LinearRegression model is not sensitive to the scale of the continuous variables. We will, however, still keep the scaled data for the next experiments.

### Experiment 2: Lasso Regression

In [ ]:
from sklearn.linear_model import Lasso

In [ ]:
lasso = Lasso()

params = {
    'alpha': np.logspace(-4, 0, 100),
    'max_iter': [15000, 20000, 30000],
    'tol': [0.0001, 0.001],
    'selection': ['cyclic', 'random'],
}

grid = GridSearchCV(lasso, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
lasso_best = Lasso(**best_params)
lasso_best.fit(Xtrain, ytrain)

In [ ]:
scores_lasso = cross_val_score(lasso_best, Xtrain, ytrain, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
scores_lasso = np.sqrt(-scores_lasso)
print(f'MEAN: {scores_lasso.mean():.2f} +/- {scores_lasso.std():.2f}')
error_percent_lasso = 100 * (10**scores_lasso.mean() - 1)
std_percent_lasso = 100 * (10**scores_lasso.std() - 1)
print(f'Average error is {error_percent_lasso:.2f}%')
print(f'Standard deviation of error is {std_percent_lasso:.2f}%')

path = DATA_DIR / 'processed' / 'models_log.csv'

with open(path, 'a') as f:
    f.write(f'Lasso,{error_percent_lasso:.2f},{std_percent_lasso:.2f}\n')

### Experiment 3:  Random Forest Regression

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor()

params = {
    'n_estimators': [100, 600, 1200, 1800],
    'min_samples_split': [3],
    'max_features': ['auto', 'sqrt'],
    'verbose': [0, 1, 2, 3],
    'max_depth': [None],
    'bootstrap': [False],
    'min_weight_fraction_leaf': [0.0],
    'min_samples_leaf': [1],
}

grid = GridSearchCV(rf, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
rf_best = RandomForestRegressor(**best_params)
rf_best.fit(Xtrain, ytrain)

In [ ]:
scores_rf = cross_val_score(rf_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_rf = np.sqrt(-scores_rf)
error_percent_rf = 100 * (10**scores_rf.mean() - 1)
std_percent_rf = 100 * (10**scores_rf.std() - 1)
print(f'Average error is {error_percent_rf:.2f}%')
print(f'Standard deviation of error is {std_percent_rf:.2f}%')

path = DATA_DIR / 'processed' / 'models_log.csv'

with open(path, 'a') as f:
    f.write(f'Random Forest,{error_percent_rf:.2f},{std_percent_rf:.2f}\n')

### Experiment 4:  Gradient Boosting Regression

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

In [ ]:
gbr = GradientBoostingRegressor()

params = {
 'alpha': 0.9, 
 'criterion': 'friedman_mse',
 'learning_rate': [0.1, 0.5, 0.01], 
 'max_depth': [3, 4, 5], 
 'max_features': 'sqrt', 
 'min_samples_leaf': [2,3], 
 'n_estimators': [1600,200], 
 'subsample': [0.9, 0.8, 0.7, 0.6], 
 'verbose': [0,1,2], 
 'warm_start': [True, False]
}

grid = GridSearchCV(gbr, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
with open('best_params.txt', 'w') as f:
    f.write(str(best_params))
print(best_params)

In [ ]:
gbr_best = GradientBoostingRegressor(**best_params)
gbr_best.fit(Xtrain, ytrain)

In [ ]:
scores_gbr = cross_val_score(gbr_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_gbr = np.sqrt(-scores_gbr)
print(f'MEAN: {scores_gbr.mean():.2f} +/- {scores_gbr.std():.2f}')
error_percent_gbr = 100 * (10**scores_gbr.mean() - 1)
std_percent_gbr = 100 * (10**scores_gbr.std() - 1)
print(f'Average error is {error_percent_gbr:.2f}%')
print(f'Standard deviation of error is {std_percent_gbr:.2f}%')

path = DATA_DIR / 'processed' / 'models_log.csv'

with open(path, 'a') as f:
    f.write(f'Gradient Boosting,{error_percent_gbr:.2f},{std_percent_gbr:.2f}\n')

### Experiment 5:  KNN Regression

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
knn = KNeighborsRegressor()

params = {
    'n_neighbors': [5, 6, 7, 8],
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'p': [1, 2, 3],
    'leaf_size': [30, 40, 50]
}

grid = GridSearchCV(knn, params, cv=5, verbose=1, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
knn_best = KNeighborsRegressor(**best_params)
knn_best.fit(Xtrain, ytrain)

In [ ]:
scores_knn = cross_val_score(knn_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_knn = np.sqrt(-scores_knn)
error_percent_knn = 100 * (10**scores_knn.mean() - 1)
std_percent_knn = 100 * (10**scores_knn.std() - 1)
print(f'Average error is {error_percent_knn:.2f}%')
print(f'Standard deviation of error is {std_percent_knn:.2f}%')

path = DATA_DIR / 'processed' / 'models_log.csv'

with open(path, 'a') as f:
    f.write(f'knn,{error_percent_knn:.2f},{std_percent_knn:.2f}\n')

### Experiment 6: ElasticNetCV

In [ ]:
from sklearn.linear_model import ElasticNetCV

In [ ]:
elnet = ElasticNetCV()

params = {
    'l1_ratio': [0.6, 0.3, 0.2, 0.1],
    'eps': [1e-2, 1e-3, 1e-4, 1e-5, 1e-6, 1e-7, 1e-8],
    'n_alphas': [100, 150, 200],
    'copy_X': [True, False],
    'verbose': [0,1,2]
}

grid = GridSearchCV(elnet, params, cv=5, n_jobs=-1)

In [ ]:
grid.fit(Xtrain, ytrain)

In [ ]:
best_params = grid.best_params_
print(best_params)

In [ ]:
elnet_best = ElasticNetCV(**best_params)
elnet_best.fit(Xtrain, ytrain)

In [ ]:
scores_elnet = cross_val_score(elnet_best, Xtrain, ytrain, cv=10, scoring='neg_mean_squared_error', n_jobs=-1)
scores_elnet = np.sqrt(-scores_elnet)
error_percent_elnet = 100 * (10**scores_elnet.mean() - 1)
std_percent_elnet = 100 * (10**scores_elnet.std() - 1)
print(f'Average error is {error_percent_elnet:.2f}%')
print(f'Standard deviation of error is {std_percent_elnet:.2f}%')

path = DATA_DIR / 'processed' / 'models_log.csv'

with open(path, 'a') as f:
    f.write(f'Elastic Net,{error_percent_elnet:.2f},{std_percent_elnet:.2f}\n')

### Experiment 7: Regression Stack

### Retraining of best model on the whole dataset

In [ ]:
best_model = linear_scaled_model

In [ ]:
best_model.fit(X, y)